In [ ]:
# Only enable critical logging (Optional)
import os
os.environ["PYCARET_CUSTOM_LOGGING_LEVEL"] = "CRITICAL"

In [ ]:
def what_is_installed():
    from pycaret import show_versions
    show_versions()

try:
    what_is_installed()
except ModuleNotFoundError:
    !pip install pycaret
    what_is_installed()

In [ ]:
import time
import numpy as np
import pandas as pd

from pycaret.datasets import get_data
from pycaret.time_series import TSForecastingExperiment

In [ ]:
#Import All Library
from pandas import DataFrame
from pandas import Series
from pandas import concat
from pandas import read_csv
from datetime import datetime
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from math import sqrt
from matplotlib import pyplot
import pandas as pd
import numpy as np
import math
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
!pip install timeseries-cv
import tsxv
import seaborn as sns
from sklearn.model_selection import TimeSeriesSplit
from keras.callbacks import EarlyStopping

In [ ]:
# load time series dataset
#df = read_csv('/kaggle/input/data-emas-skripsi/gold prices full.csv', header=0, index_col=1).drop("Unnamed: 0", axis=1)
df = pd.read_csv('/kaggle/input/data-emas-skripsi/gold prices.csv',index_col='index',parse_dates=True).drop("Unnamed: 0", axis=1)
df.tail()

In [ ]:
#df.plot()
pyplot.xticks(rotation=30)
#plt.figure(figsize=(20, 12))
plt.title('Gold Price 2003-2023')
plt.plot(df['value'],color="g")
plt.xlabel('Year')
plt.ylabel('Price (USD)')
#pyplot.show()

In [ ]:
# We want to forecast the next 12 months of data and we will use 3 fold cross-validation to test the models.
fh = 7*52*2 # or alternately fh = np.arange(1,13)
fold = 11

In [ ]:
from sktime.forecasting.all import *
from sktime.forecasting.model_evaluation import evaluate

exp = TSForecastingExperiment()

#### Add a custom Cross Validation Splitter with a
####  Step length = 6, Forecast Horizon = 24, Window Length = 48
my_custom_cv = SlidingWindowSplitter(fh=np.arange(1, ((7*52*2)+1)), window_length=(7*52*8), step_length=(52*7))
#### exp.setup(data=df, session_id=42, fold_strategy=my_custom_cv)

In [ ]:
s = list(my_custom_cv.split(df))
#print(f"First split of SlidingWindowSplitter: {s[0]}")

In [ ]:
print(f"Train Data 1: {s[0][0]}")
print(f"Test Data 1: {s[0][1]}")

In [ ]:
#Train Length
len(s[0][0]) #2912

#Test Length
len(s[0][1]) #728

In [ ]:
s[0][0][0]
s[0][0][2911]
s[0][1][0]
s[0][1][727]

****

In [ ]:
def folds(df, len_slices, batch_size, epoch, learning_rate, optimizer, earlystop):
    all_pred=[]
    all_rmse=[]
    all_mape=[]
    for slices in range(0,len_slices):
        values = df.values
        #Train Test
        scaler = StandardScaler()
        scaled_data = scaler.fit_transform(values)

        train = scaled_data[0: len(s[slices][0]), :]

        x_train = []
        y_train = []

        for i in range((52*7), len(s[slices][0])):
            x_train.append(train[i-(52*7):i, 0])
            y_train.append(train[i, 0])

        x_train, y_train = np.array(x_train), np.array(y_train)

        x_train = np.reshape(x_train, (x_train.shape[0], x_train.shape[1], 1))

        test_y = scaled_data[s[slices][1][0]-(52*7):s[slices][1][727], :] #ubah bagian ini ke semua slices

        x_test = []
        y_test = values[s[slices][1][0]:s[slices][1][727]]

        for i in range((52*7), len(test_y)):
          x_test.append(test_y[i-(52*7):i, 0])

        x_test = np.array(x_test)
        x_test = np.reshape(x_test, (x_test.shape[0], x_test.shape[1], 1))

        model = keras.Sequential()
        model.add(layers.LSTM(50, return_sequences=True, input_shape=(x_train.shape[1], 1)))
        model.add(layers.LSTM(50, return_sequences=False))
        model.add(layers.Dense(1))
        es = EarlyStopping(monitor='loss', patience = 7) #mode='min', verbose=1, patience=200)
        if optimizer == "adam":
            opt = keras.optimizers.Adam(learning_rate=learning_rate)
            model.compile(optimizer=opt, loss='mean_squared_error')
            if earlystop == "es":
                model.fit(x_train, y_train, batch_size= batch_size, epochs=epoch, callbacks=[es])
            else:
                model.fit(x_train, y_train, batch_size= batch_size, epochs=epoch)
        else:
            opt = keras.optimizers.RMSprop(learning_rate=learning_rate)
            model.compile(optimizer=opt, loss='mean_squared_error')
            if earlystop == "es":
                model.fit(x_train, y_train, batch_size= batch_size, epochs=epoch, callbacks=[es])
            else:
                model.fit(x_train, y_train, batch_size= batch_size, epochs=epoch)

        predictions = model.predict(x_test)
        predictions = scaler.inverse_transform(predictions)
        rmse = sqrt(mean_squared_error(y_test, predictions))
        mape = mean_absolute_percentage_error(y_test, predictions)*100
        
        all_pred.append(predictions)
        all_rmse.append(rmse)
        all_mape.append(mape)

    return all_pred, all_rmse, all_mape

In [ ]:
from statistics import mean
from statistics import variance
from statistics import stdev

def mean_stdev_acc(lst): 
    rmse_mean = round(mean(lst[1]),4)
    rmse_stdev = round(stdev(lst[1]),4)
    print("RMSE Average =", round(rmse_mean, 4))
    print("RMSE St.Dev =", round(rmse_stdev, 4))
    mape_mean = round(mean(lst[2]),4)
    mape_stdev = round(stdev(lst[2]),4)
    print("MAPE Average =", round(mape_mean, 4))
    print("MAPE St.Dev =", round(mape_stdev, 4))
    return rmse_mean, rmse_stdev, mape_mean, mape_stdev

In [ ]:
s1 = folds(df, len_slices = 12, batch_size = 56, epoch = 100, 
           learning_rate = 0.01, optimizer = "adam", earlystop = "es")
mean_stdev_acc(s1)

In [ ]:
s2 = folds(df, len_slices = 12, batch_size = 56, epoch = 500, 
           learning_rate = 0.01, optimizer = "adam", earlystop = "es")
mean_stdev_acc(s2)

In [ ]:
s3 = folds(df, len_slices = 12, batch_size = 56, epoch = 1000, 
           learning_rate = 0.01, optimizer = "adam", earlystop = "es")
mean_stdev_acc(s3)

In [ ]:
s4 = folds(df, len_slices = 12, batch_size = 56, epoch = 100, 
           learning_rate = 0.001, optimizer = "adam", earlystop = "es")
mean_stdev_acc(s4)

In [ ]:
s5 = folds(df, len_slices = 12, batch_size = 56, epoch = 500, 
           learning_rate = 0.001, optimizer = "adam", earlystop = "es")
mean_stdev_acc(s5)

In [ ]:
s6 = folds(df, len_slices = 12, batch_size = 56, epoch = 1000, 
           learning_rate = 0.001, optimizer = "adam", earlystop = "es")
mean_stdev_acc(s6)

In [ ]:
s7 = folds(df, len_slices = 12, batch_size = 56, epoch = 100, 
           learning_rate = 0.0001, optimizer = "adam", earlystop = "es")
mean_stdev_acc(s7)

In [ ]:
s8 = folds(df, len_slices = 12, batch_size = 56, epoch = 500, 
           learning_rate = 0.0001, optimizer = "adam", earlystop = "es")
mean_stdev_acc(s8)

In [ ]:
s9 = folds(df, len_slices = 12, batch_size = 56, epoch = 1000, 
           learning_rate = 0.0001, optimizer = "adam", earlystop = "es")
mean_stdev_acc(s9)